In [1]:
# Here we import everything we need for the project

%matplotlib inline
import os, time
import csv

# pytorch
import torch
from torch.nn import Module, Conv2d, MaxPool2d, Linear
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn.functional as F

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd

# Sklearn
from sklearn.model_selection import train_test_split # Helps with organizing data for training
from sklearn.metrics import confusion_matrix, classification_report # Helps present results as a confusion-matrix

## Loading Data

This project uses the [Hand Gesture Recognition Database](https://www.kaggle.com/gti-upm/leapgestrecog/version/1) (citation below) available on Kaggle. It contains 20000 images with different hands and hand gestures. There is a total of 10 hand gestures of 10 different people presented in the dataset. There are 5 female subjects and 5 male subjects.
The images were captured using the Leap Motion hand tracking device.

>Hand Gesture | Label used
>--- | ---
> Thumb down | 0
> Palm (Horizontal) | 1
> L | 2
> Fist (Horizontal) | 3
> Fist (Vertical) | 4
> Thumbs up | 5
> Index | 6
> OK | 7
> Palm (Vertical) | 8
> C | 9

Table 1 - Classification used for every hand gesture.


T. Mantecón, C.R. del Blanco, F. Jaureguizar, N. García, “Hand Gesture Recognition using Infrared Imagery Provided by Leap Motion Controller”, Int. Conf. on Advanced Concepts for Intelligent Vision Systems, ACIVS 2016, Lecce, Italy, pp. 47-57, 24-27 Oct. 2016. (doi: 10.1007/978-3-319-48680-2_5)  

Overview:
- Load images
- Some validation
- Preparing the images for training
- Use of train_test_split

In [2]:
# Unzip images, ignore this cell if files are already in the workspace
#!unzip leapGestRecog.zip

## Writing CSV - Kaggle Dataset
Since this dataset doesn't come with a nice csv, write one ourselves to make loading the data easier later

In [3]:
header = ["path_to_file", "class/GT"]
csv_path = "kaggle_images.csv"
# We need to get all the paths for the images to later load them
imagepaths = []
root = "./leapgestrecog"

for dirname, dirs, files in os.walk(root):
    for fname in files:
        path = os.path.join(dirname, fname)
        if path.endswith(".png"):
            imagepaths.append(path)

print(len(imagepaths)) # If > 0, then a PNG image was loaded
categories = [fpath.split("/")[4] for _, fpath in enumerate(imagepaths)]
gt = [category.split("_")[0] for _, category in enumerate(categories)]
with open(csv_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(header)
    for i, fpath in enumerate(imagepaths):
        writer.writerow([fpath, gt[i]])

0


## Writing CSV - Kaggle Dataset
Since this dataset doesn't come with a nice csv, write one ourselves to make loading the data easier later
Note: directly setting filepaths to the pre-binarized images so we don't need to perform this operation ourselves

In [4]:
header = ["path_to_file", "GT"]
csv_path = "./data/asl/asl_images.csv"
imagepaths = []
fnames = []
# these are 400x400 BW images
root = "./data/asl/asl_data/binary_frames_rotated"

for dirname, dirs, files in os.walk(root):
    for fname in files:
        if fname.endswith(".png"):
            fnames.append(fname)
        path = os.path.join(dirname, fname)
        if path.endswith(".png"):
            imagepaths.append(path)

gt = [fname.split('_')[0] for fname in fnames]
with open(csv_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(header)
    for i, fpath in enumerate(imagepaths):
        writer.writerow([fpath, gt[i]])

## ASL Helpers
Mostly used to create a somewhat balanced train/test split, and to visualize counts of every class

In [5]:
def make_asl_count_df(y):
    counts = np.zeros(26)
    y = np.array(y)
    for i in range(26):
        counts[i] = np.sum(np.where(y == i, 1, 0))
    idx = [chr(i) for i in range(97, 123)]
    columns=["Count"]
    df = pd.DataFrame(counts, index=idx, columns=columns)
    return df

def y_as_np_arr(dataset):
    return np.array([sample['y'] for sample in dataset])


def make_asl_train_test_split(dataset, counts_df, split_ratio=0.75, train_csv="./data/asl/train_asl.csv", test_csv="./data/asl/test_asl.csv"):
    header = ["path_to_file", "GT"]
    print(0)
    train = []
    test = []
    np.random.seed(0)
    print(1)
    train_counts, test_counts = [int(np.ceil(split_ratio*counts_df.iloc[idx, 0])) for idx in range(counts_df.shape[0])], [int(np.floor((1-split_ratio)*counts_df.iloc[idx, 0])) for idx in range(counts_df.shape[0])]
#     y = y_as_np_arr(dataset)
    print(2)
    for idx in range(counts_df.shape[0]):
        #curr_train_selections = []
        #curr_test_selections = []
        only_class_locs = np.where(y==idx)[0]
        train_idxes = np.random.choice(only_class_locs, size=train_counts[idx], replace=False)
        for data_idx in only_class_locs:
            sample = dataset[data_idx]
            if data_idx in train_idxes:
                #curr_train_selections.append(sample['fname'])
                train.append(sample['fname'])
            else:
                #curr_test_selections.append(sample['fname'])
                test.append(sample['fname'])
        #train.append(curr_train_selections)
        #test.append(curr_test_selections)
    
    print(3)
#     train_lasts = [fname.split('/')[-1] for fname in train]
    train_gt = [ os.path.split(fname)[-1].split('_')[0] for fname in train]
#     test_lasts = [fname.split('/')[-1] for fname in test]
    test_gt = [os.path.split(fname)[-1].split('_')[0] for fname in test]
    print(train_gt[:5])
    print(test_gt[:5])
    with open(train_csv, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(header)
        for i, fpath in enumerate(train):
            writer.writerow([fpath, train_gt[i]])
    with open(test_csv, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(header)
        for i, fpath in enumerate(test):
            writer.writerow([fpath, test_gt[i]])
    return train, test



#plt.imshow(np.squeeze(test[0][0]['image'].numpy()))

In [16]:
# for sample in asl_dataset:
#     print(sample['y'])

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


KeyboardInterrupt: 

0
1
2
3
['a', 'a', 'a', 'a', 'a']
['a', 'a', 'a', 'a', 'a']


In [20]:
# print(test)

4320
12960


## Dataloaders
To facilitate using pytorch to build our cnn, we write a custom DataLoader class. This allows for on-demand loading of images, which are used to train our cnn.

In [22]:
class KaggleHandDetectionDataset(Dataset):
    """Custom loader for the Kaggle Hand Detection Dataset"""
    
    def __init__(self, csv_file, transforms=None):
        """
        Args:
            csv_file (string): Path to the csv file with image filepaths and gt classes
            transforms (callable, optional): Optional transforms to be applied on a sample
        """
        self.images_frame = pd.read_csv(csv_file)
        #print(self.images_frame)
        self.transforms = transforms
        
    def __len__(self):
        return len(self.images_frame)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        img_name = self.images_frame.iloc[idx, 0]
        img = cv2.imread(img_name)
        y = int(self.images_frame.iloc[idx, 1])-1
        sample = {'image' : img, 'y' : y, 'fname' : img_name}
        
        if len(self.transforms) > 0:
            for _, transform in enumerate(self.transforms):
                sample = transform(sample)
        return sample


In [23]:
class AslGestureDataset(Dataset):
    """Custom loader for the Kaggle Hand Detection Dataset"""
    
    def __init__(self, csv_file, transforms=None):
        """
        Args:
            csv_file (string): Path to the csv file with image filepaths and gt classes
            transforms (callable, optional): Optional transforms to be applied on a sample
        """
        self.images_frame = pd.read_csv(csv_file)
        self.transforms = transforms
        
    def __len__(self):
        return len(self.images_frame)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        img_name = self.images_frame.iloc[idx, 0]
        #print(img_name)
        img = cv2.imread(img_name)
        y = (self.images_frame.iloc[idx, 1])
        # normalize to 0-26 for classes (missing j and z b/c dynamic)
        y = ord(y)- ord('a')
        sample = {'image' : img, 'y' : y, 'fname' : img_name}
        
        if len(self.transforms) > 0:
            for _, transform in enumerate(self.transforms):
                sample = transform(sample)
        return sample

In [24]:
class Rescale(object):
    """Used to rescale an image to a given size. Useful for the CNN
    
    Args:
        output_size (tuple or int): Desired output size after rescaling. If tuple, output is matched to output_size.
        If int smaller of width/height is matched to output_size, keeping aspect ratio the same.
    """
    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size
        
    def __call__(self, sample):
        y = sample['y']
        image = sample['image']
        
        h, w = image.shape[:2]
        if isinstance(self.output_size, int):
            if h > w:
                new_h, new_w = self.output_size * h // w, self.output_size
            else:
                new_h, new_w = self.output_size, self.output_size * w // h
        else:
            new_h, new_w = self.output_size
        
        img = cv2.resize(image, (new_h, new_w))
        #print(f"after resize: {img.shape}")
        return {'image' : img, 'y' : y, 'fname' : sample['fname']}
    
class Recolor(object):
    """Used to recolor an image using cv2
    
    Args:
        flag (cv2.COLOR_): color to swap to
    """
    def __init__(self, color):
        self.color = color
    
    def __call__(self, sample):
        y = sample['y']
        image = sample['image']
        #print(f"before recolor: {image.shape}")
        
        # cvtColor to gray drops the damned channel dimension but we need it
        img_cvt = cv2.cvtColor(image, self.color)
        # fucking hack an extra dim to appease pytorch's bitchass
        img_cvt = np.expand_dims(img_cvt, axis=-1)
        #print(f"after exansion: {img_cvt.shape}")
        return {'image' : img_cvt, 'y' : y, 'fname' : sample['fname']}

class ToTensor(object):
    """Convert ndarrays to pytorch Tensors"""
    
    def __call__(self, sample):
        y = sample['y']
        image = sample['image']
        
        # swap color axis b/c 
        # numpy img: H x W x C
        # torch img: C x H x W
        image = image.transpose((2, 0, 1))
        return {'image' : torch.from_numpy(image), 'y' : y, 'fname' : sample['fname']}

In [25]:
def prediction_to_class_str(pred):
    classes = {0 : "palm", 1 : "L", 2 : "fist", 3 : "fist_moved", 4 : "thumb", 5 : "index", 6 : "ok", 7 : "palm_moved", 8 : "c", 9 : "down"}
    return classes[pred]

def classify_arbitrary_image(model, img):
    img_type = type(img)
    print(img_type)
    if img_type == torch.Tensor:
        print("tensor")
        img = img.float()
        img = img.unsqueeze(1)
    elif img_type == np.ndarray:
        print("np array")
        img = np.expand_dims(img, 1)
    else:
        print("error: something other than a torch.Tensor or an np.ndarray was passed as img")
    prediction = model(img)
    prediction = prediction.data.numpy()
    y_hat = np.argmax(prediction, axis=1)
    return prediction_to_class_str(y_hat[0])

def classify_many_images(model, imgs):
    # for now, assuming imgs is a list of images that are either np.ndarrays or torch.Tensors
    # labels will be given back in order images were given
    predictions = []
    for img in imgs:
        predictions.append(classify_arbitrary_image(model, img))
    return predictions

In [26]:
class HandNNModel(Module):
    def __init__(self):
        super().__init__()
        
        # input shape = (32, 256, 256) - (batch_size, w, h) from dataloader
        self.conv1 = Conv2d(1, 32, kernel_size=5) # output shape: (252, 252, 32)
        self.pool1 = MaxPool2d(2) # output shape: (121, 121, 32)
        self.conv2 = Conv2d(32, 64, kernel_size=3) # output shape: (119, 119, 64)
        self.pool2 = MaxPool2d(2) # output shape: (59, 59, 64) - torch uses floor by default
        self.conv3 = Conv2d(64, 64, kernel_size=3) # output shape: (57, 57, 64)
        self.pool3 = MaxPool2d(2) # output shape: (28, 28, 64)
        self.fc1 = Linear(28*28*64, 128) # output shape: (28*28*64, 128)
        self.fc2 = Linear(128, 10)
        self.activation = torch.nn.ReLU()
        
    def forward(self, X):
        X = self.activation(self.conv1(X))
        X = self.pool1(X)
        X = self.activation(self.conv2(X))
        X = self.pool2(X)
        X = self.activation(self.conv3(X))
        X = self.pool3(X)
        X = torch.flatten(X, 1) # flatten with start_dim = 1
        X = self.fc1(X)
        X = self.fc2(X)
        output = F.softmax(X)
        return output

In [27]:
class AslNNModel(Module):
    # same structure as HandNNModel, need to change dimensions
    def __init__(self):
        super().__init__()
        
        # input shape = (64, 400, 400) - (batch_size, w, h) from dataloader
        self.conv1 = Conv2d(1, 32, kernel_size=5) # output shape: (496, 496, 32)
        self.pool1 = MaxPool2d(2) # output shape: (198, 198, 32)
        self.conv2 = Conv2d(32, 64, kernel_size=3) # output shape: (196, 196, 64)
        self.pool2 = MaxPool2d(2) # output shape: (98, 98, 64) - torch uses floor by default
        self.conv3 = Conv2d(64, 64, kernel_size=3) # output shape: (96, 96, 64)
        self.pool3 = MaxPool2d(2) # output shape: (48, 48, 64)
        self.fc1 = Linear(48*48*64, 128) # output shape: (48*48*64, 128)
        self.fc2 = Linear(128, 26) # 24 possible output classes, but it goes up to idx 26: CUDA screams otherwise, so here we are
        self.activation = torch.nn.ReLU()
        
    def forward(self, X):
        X = self.activation(self.conv1(X))
        X = self.pool1(X)
        X = self.activation(self.conv2(X))
        X = self.pool2(X)
        X = self.activation(self.conv3(X))
        X = self.pool3(X)
        X = torch.flatten(X, 1) # flatten with start_dim = 1
        X = self.fc1(X)
        X = self.fc2(X)
        output = F.softmax(X)
        return output

In [28]:
resize = Rescale((256,256))
recolor = Recolor(cv2.COLOR_BGR2GRAY)
to_tensor = ToTensor()
transforms = [resize, recolor, to_tensor]

#hand_dataset = KaggleHandDetectionDataset(csv_file="kaggle_images.csv", transforms=transforms)

# for some reason, even with the binarization, there's 3 channels, but we only want one - so, use recolor
asl_dataset = AslGestureDataset(csv_file="./data/asl/asl_images.csv", transforms=[recolor, to_tensor])
train_asl_dataset = AslGestureDataset(csv_file="./data/asl/train_asl.csv", transforms=[recolor, to_tensor])
print(len(train_asl_dataset))
# df = pd.read_csv("kaggle_images.csv")

sample = asl_dataset[0]
print()
print(sample['image'].shape)
print(sample['y'])
print(sample)
#sample = hand_dataset[0]
#print()
#print(sample['image'].shape)
#print(sample['y'])
#print(sample)

12960

torch.Size([1, 400, 400])
0
{'image': tensor([[[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]], dtype=torch.uint8), 'y': 0, 'fname': './data/asl/asl_data/binary_frames_rotated\\a_100-19.png'}


## Splitting Data

In [ ]:
asl_dataset = AslGestureDataset(csv_file="./data/asl/asl_images.csv", transforms=[recolor, to_tensor])

In [ ]:
y = y_as_np_arr(asl_dataset)

In [ ]:
df = make_asl_count_df(y)

In [ ]:
train, test = make_asl_train_test_split(asl_dataset, df)

In [ ]:
lasts = [fname.split('/')[-1] for fname in test]
gt = [fname.split('_')[0] for fname in lasts]
print(len(gt))
train_asl_dataset = AslGestureDataset(csv_file="./data/asl/train_asl.csv", transforms=[recolor, to_tensor])
print(len(train_asl_dataset))

In [29]:
"""
sample = asl_dataset[0]
print()
print(sample['image'].shape)
print(sample['y'])
print(sample)
sample = hand_dataset[0]
img = sample['image']
#recolor = Recolor(cv2.COLOR_BGR2GRAY)
#as_gray = recolor(sample)
#scale = Rescale((256, 256))
#tns = ToTensor()
#as_tns = tns(sample)
#print(type(as_tns['image']))
#new_img = scale(sample)
print(sample['image'])
# complains b/c the gpu owns it, not the cpu since it's a tensor
cv2.imshow('original', sample['image'])
cv2.waitKey(2000)
#cv2.imshow('recolored', as_gray['image'])
#cv2.waitKey(2000)
#cv2.imshow('rescaled', new_img['image'])
#cv2.waitKey(3000)
cv2.destroyAllWindows()
"""


"\nsample = asl_dataset[0]\nprint()\nprint(sample['image'].shape)\nprint(sample['y'])\nprint(sample)\nsample = hand_dataset[0]\nimg = sample['image']\n#recolor = Recolor(cv2.COLOR_BGR2GRAY)\n#as_gray = recolor(sample)\n#scale = Rescale((256, 256))\n#tns = ToTensor()\n#as_tns = tns(sample)\n#print(type(as_tns['image']))\n#new_img = scale(sample)\nprint(sample['image'])\n# complains b/c the gpu owns it, not the cpu since it's a tensor\ncv2.imshow('original', sample['image'])\ncv2.waitKey(2000)\n#cv2.imshow('recolored', as_gray['image'])\n#cv2.waitKey(2000)\n#cv2.imshow('rescaled', new_img['image'])\n#cv2.waitKey(3000)\ncv2.destroyAllWindows()\n"

In [30]:
dataloader = DataLoader(asl_dataset, batch_size=4)
for i_batch, sample_batched in enumerate(dataloader):
    print(i_batch, sample_batched['image'].size())
    if i_batch == 3:
        break

0 torch.Size([4, 1, 400, 400])
1 torch.Size([4, 1, 400, 400])
2 torch.Size([4, 1, 400, 400])
3 torch.Size([4, 1, 400, 400])


## Train the model using the Custom Dataloader
Below, we will actually train our CNN model

In [31]:
use_cuda = torch.cuda.is_available()
print(use_cuda)

True


In [134]:
torch.cuda.empty_cache()

In [142]:
import gc
for obj in gc.get_objects():
    try:
        if torch.is_tensor(obj) or (hasattr(obj, 'data') and torch.is_tensor(obj.data)):
            print(type(obj), obj.size())
            print(gc.get_referrers(obj))
    except Exceptioneption e:
        print(e)

C:\Users\wills\AppData\Local\Programs\Python\Python36\lib\site-packages\torch\distributed\distributed_c10d.py:144: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn("torch.distributed.reduce_op is deprecated, please use "
C:\Users\wills\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  after removing the cwd from sys.path.
C:\Users\wills\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime instead.
  after removing the cwd from sys.path.


<class 'torch.Tensor'> torch.Size([4])
<class 'torch.Tensor'> torch.Size([4, 1, 400, 400])
<class 'torch.nn.parameter.Parameter'> torch.Size([32, 1, 5, 5])
<class 'torch.nn.parameter.Parameter'> torch.Size([32])
<class 'torch.nn.parameter.Parameter'> torch.Size([64, 32, 3, 3])
<class 'torch.nn.parameter.Parameter'> torch.Size([64])
<class 'torch.nn.parameter.Parameter'> torch.Size([64, 64, 3, 3])
<class 'torch.nn.parameter.Parameter'> torch.Size([64])
<class 'torch.nn.parameter.Parameter'> torch.Size([128, 147456])
<class 'torch.nn.parameter.Parameter'> torch.Size([128])
<class 'torch.Tensor'> torch.Size([48])
<class 'torch.Tensor'> torch.Size([48])
<class 'torch.Tensor'> torch.Size([48, 1, 400, 400])
<class 'torch.Tensor'> torch.Size([48, 26])
<class 'torch.Tensor'> torch.Size([])
<class 'torch.nn.parameter.Parameter'> torch.Size([26, 128])
<class 'torch.nn.parameter.Parameter'> torch.Size([26])
<class 'torch.Tensor'> torch.Size([32, 1, 5, 5])
<class 'torch.Tensor'> torch.Size([32])
<

In [135]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
batch = 20
if use_cuda:
    params = {'batch_size' : batch, 'shuffle': True, 'num_workers' : 0, 'pin_memory' : True}
else:
    params = {'batch_size' : batch, 'shuffle': True, 'num_workers' : 0}
    

In [136]:
test_loss = []
train_loss = []

In [137]:
max_epochs = 1 

# test_asl_dataset = AslGestureDataset(csv_file="./data/asl/test_asl.csv", transforms=[recolor, to_tensor])

model = AslNNModel()
# move to GPU
if use_cuda:
    model.cuda()

# batch_size, num_channels, w, h
#random_data = torch.rand((1, 1, 256, 256))

#result = model(random_data)

criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

#imgs, labels = (next(iter(data_loader)))
start = time.time()
for epoch in range(max_epochs):
    print(f"start epoch {epoch}")
    running_loss_train = 0.0
    epoch_start_train = time.time()
    count_train = 0
    
    data_loader = DataLoader(train_asl_dataset, **params)
    for i, data in enumerate(data_loader):
        imgs, labels = data['image'], data['y']
        # move to GPU
        imgs, labels = imgs.cuda(), labels.cuda()
        imgs = imgs.float()
        optimizer.zero_grad()
        
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss_train += loss.item()
        count_train = i + 1
        if i %  50 == 0:
            print(f"epoch {epoch} training ({count_train * batch}/17,280): avg running loss = {running_loss_train / count_train}")
            print(f"epoch {epoch} training ({count_train * batch}/17,280): time passed = {time.time() - epoch_start_train} seconds")
    
    print(f"epoch {epoch}: training final loss = {running_loss_train/count_train}")
    train_loss.append(running_loss_train/count_train)
    epoch_end_train = time.time()
    print(f"epoch {epoch} training runtime = {epoch_end_train - epoch_start_train}")
    
    running_loss_test = 0.0
    epoch_start_test = time.time()
    count_test = 0
#     test_data_loader = DataLoader(test_asl_dataset, **params)
#     print(f"epoch {epoch}: final loss = {running_loss/count}")
    for i, data in enumerate(test_data_loader):
        imgs, labels = data['image'], data['y']
        # move to GPU
        imgs, labels = imgs.cuda(), labels.cuda()
        imgs = imgs.float()
        
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        
        running_loss_test += loss.item()
        count = i + 1
        if i %  50 == 0:
            print(f"epoch {epoch} testing ({count_test * batch}/17,280): avg running loss = {running_loss_test / count_test}")
            print(f"epoch {epoch} testing ({count_test * batch}/17,280): time passed = {time.time() - epoch_start_test} seconds")
    print(f"epoch {epoch}: testing final loss = {running_loss_test/count_test}")
    train_loss.append(running_loss_test/count_test)
    epoch_end_test = time.time()
    print(f"epoch {epoch} testing runtime = {epoch_end_test - epoch_start_test}")
    
end = time.time()
print(f"Total training time = {end - start}")

# make sure to save the model so we don't need to train again
save = True
if save:
    save_path = "./data/asl/asl_train_no_tl_same_model.pkl"
    torch.save(model.state_dict(), save_path)

start epoch 0


RuntimeError: CUDA out of memory. Tried to allocate 384.00 MiB (GPU 0; 8.00 GiB total capacity; 5.95 GiB already allocated; 0 bytes free; 6.43 GiB reserved in total by PyTorch)

In [138]:
#print(model.named_parameters)

model_params = model.parameters()

## Saving Trained Model

In [144]:
save_path = "./trained_test_cnn.pkl"
torch.save(model.state_dict(), save_path)

## Loading the model
Load the model from disk later for evaluation

In [53]:
loaded_model = AslNNModel()
loaded_model.load_state_dict(torch.load(save_path))

<All keys matched successfully>

## Confusion Matrix and Classification Report (from sklearn)


In [55]:
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report
def make_stats(y, y_hat, num_classes=10):
    cm = confusion_matrix(y, y_hat)
    cm_df = pd.DataFrame(cm, columns=[str(i) for i in range(num_classes)])
    report = classification_report(y, y_hat)
    return cm_df, report

In [56]:
test_asl_dataset = AslGestureDataset(csv_file="./data/asl/test_asl.csv", transforms=[recolor, to_tensor])

In [57]:
print(params)

{'batch_size': 48, 'shuffle': True, 'num_workers': 0, 'pin_memory': True}


# Training stats

In [93]:
total_samples = 0
total_misclass = 0
all_y = np.array([], dtype=np.uint8)
all_y_hat = np.array([], dtype=np.uint8)

if use_cuda:
    model = model.cuda()
for i, sample in enumerate(data_loader):
    y = sample['y']
    y = y.data.numpy()
#     images = (sample['image'])
#     images = images.float()
    images = (sample['image'])
    if use_cuda:
        images = images.cuda()
    images = images.float()
    
#     model = model.cpu()
    predictions = model(images)
    predictions = predictions.cpu()
    predictions = predictions.data.numpy()
    y_hat = np.argmax(predictions, axis=1)
    misclass = np.sum(np.where(y != y_hat, 1, 0))
    total_samples += y.shape[0]
    total_misclass += misclass
    all_y = np.append(all_y, y)
    all_y_hat = np.append(all_y_hat, y_hat)
    #print(f"all_y = {all_y}")
    #print(f"all_y_hat = {all_y_hat}")
    print(f"Number of Misclassifications = {misclass}")
    print(f"Sample acc = {(y.shape[0]-misclass)/y.shape[0]*100}")
overall_acc = (total_samples - total_misclass)/total_samples
print(f"Overall Accuracy = {overall_acc}")

C:\Users\wills\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Number of Misclassifications = 11
Sample acc = 77.08333333333334
Number of Misclassifications = 8
Sample acc = 83.33333333333334
Number of Misclassifications = 5
Sample acc = 89.58333333333334
Number of Misclassifications = 7
Sample acc = 85.41666666666666
Number of Misclassifications = 11
Sample acc = 77.08333333333334
Number of Misclassifications = 7
Sample acc = 85.41666666666666
Number of Misclassifications = 8
Sample acc = 83.33333333333334
Number of Misclassifications = 7
Sample acc = 85.41666666666666
Number of Misclassifications = 6
Sample acc = 87.5
Number of Misclassifications = 7
Sample acc = 85.41666666666666
Number of Misclassifications = 5
Sample acc = 89.58333333333334
Number of Misclassifications = 5
Sample acc = 89.58333333333334
Number of Misclassifications = 6
Sample acc = 87.5
Number of Misclassifications = 5
Sample acc = 89.58333333333334
Number of Misclassifications = 7
Sample acc = 85.41666666666666
Number of Misclassifications = 9
Sample acc = 81.25
Number of Mi

In [115]:
alphabet = [chr(l) for l in range(ord('a'), ord('z') +1)]
alphabet_reduced = [chr(l) for l in range(ord('a'), ord('z') +1)]
alphabet_reduced.remove("j")
alphabet_reduced.remove("z")

In [95]:
cm_df, report = make_stats(all_y, all_y_hat, num_classes=26)
#print(cm_df.to_markdown())
cm_df.index = alphabet
cm_df.columns = alphabet
print(cm_df.to_markdown())
for cl, lt in enumerate(alphabet):
    cl_str = f' {cl} '
    lt_str = f' {lt} '
    lt_str = ' '*(len(cl_str) - len(lt_str)) + lt_str
    report = report.replace(cl_str, lt_str)
print(report)
    
#     print(type(report))

|    |   a |   b |   c |   d |   e |   f |   g |   h |   i |   j |   k |   l |   m |   n |   o |   p |   q |   r |   s |   t |   u |   v |   w |   x |   y |   z |
|:---|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|
| a  | 487 |   2 |   1 |   7 |   2 |   4 |   1 |   1 |   1 |   0 |   0 |   0 |  15 |   4 |   4 |   0 |   0 |   0 |   1 |   4 |   0 |   4 |   0 |   0 |   2 |   0 |
| b  |   4 | 472 |   3 |   3 |   0 |  17 |   4 |   5 |   0 |   0 |   0 |   0 |   2 |   0 |   1 |   6 |   7 |   1 |   0 |   6 |   2 |   3 |   0 |   1 |   3 |   0 |
| c  |   1 |   4 | 487 |   1 |   0 |   0 |   4 |  12 |   2 |   0 |   3 |   0 |   0 |   1 |   0 |   3 |   6 |   7 |   2 |   0 |   3 |   0 |   0 |   3 |   1 |   0 |
| d  |   2 |   0 |   5 | 456 |   4 |   1 |  12 |   2 |   3 |   0 |   5 |   9 |   0 |   0 |   0 |   6 |   6 |   6 |   3 |   5 |   0 |   9 |   1 |   0 |   5 |   0 |
| e  |  19 |   0 |   1

C:\Users\wills\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\wills\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\wills\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r

|    |   a |   b |   c |   d |   e |   f |   g |   h |   i |   j |   k |   l |   m |   n |   o |   p |   q |   r |   s |   t |   u |   v |   w |   x |   y |   z |
|:---|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|
| a  | 487 |   2 |   1 |   7 |   2 |   4 |   1 |   1 |   1 |   0 |   0 |   0 |  15 |   4 |   4 |   0 |   0 |   0 |   1 |   4 |   0 |   4 |   0 |   0 |   2 |   0 |
| b  |   4 | 472 |   3 |   3 |   0 |  17 |   4 |   5 |   0 |   0 |   0 |   0 |   2 |   0 |   1 |   6 |   7 |   1 |   0 |   6 |   2 |   3 |   0 |   1 |   3 |   0 |
| c  |   1 |   4 | 487 |   1 |   0 |   0 |   4 |  12 |   2 |   0 |   3 |   0 |   0 |   1 |   0 |   3 |   6 |   7 |   2 |   0 |   3 |   0 |   0 |   3 |   1 |   0 |
| d  |   2 |   0 |   5 | 456 |   4 |   1 |  12 |   2 |   3 |   0 |   5 |   9 |   0 |   0 |   0 |   6 |   6 |   6 |   3 |   5 |   0 |   9 |   1 |   0 |   5 |   0 |
| e  |  19 |   0 |   1 |   4 | 382 |   3 |   2 |   0 |  12 |   0 |   0 |   0 |  31 |  12 |   9 |   0 |   3 |   9 |  12 |  32 |   3 |   0 |   1 |   2 |   3 |   0 |
| f  |   1 |   1 |   2 |   0 |   5 | 457 |   2 |   5 |   1 |   0 |   4 |   3 |   7 |   1 |   3 |   0 |   3 |   0 |  16 |  24 |   1 |   0 |   1 |   1 |   2 |   0 |
| g  |   6 |   7 |  14 |   7 |   5 |   7 | 417 |   8 |   2 |   0 |   4 |   1 |   0 |   1 |   1 |  16 |   0 |  10 |   0 |   5 |   0 |   5 |   7 |   3 |  14 |   0 |
| h  |   0 |   0 |   1 |   1 |   0 |   0 |   5 | 510 |   1 |   1 |   0 |   0 |   1 |   1 |   0 |   2 |   1 |   4 |   4 |   0 |   3 |   2 |   0 |   3 |   0 |   0 |
| i  |   1 |   4 |   1 |   0 |   5 |   0 |   0 |   0 | 506 |   0 |   1 |   2 |   6 |   2 |   1 |   0 |   0 |   0 |   1 |   6 |   1 |   0 |   1 |   0 |   2 |   0 |
| j  |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |
| k  |   0 |   2 |  12 |   0 |   0 |   0 |   1 |   0 |   2 |   0 | 481 |   0 |   0 |   1 |   0 |   2 |   1 |   3 |   0 |   0 |   4 |   4 |   9 |   3 |  15 |   0 |
| l  |   0 |   1 |   2 |   1 |   0 |   0 |   1 |   0 |   1 |   0 |   3 | 519 |   0 |   0 |   0 |   0 |   0 |   2 |   0 |   2 |   2 |   2 |   2 |   2 |   0 |   0 |
| m  |   6 |   6 |   0 |   0 |   7 |   1 |   2 |   1 |   8 |   0 |   2 |   1 | 424 |  24 |   6 |   2 |   0 |   3 |  19 |  23 |   1 |   0 |   0 |   4 |   0 |   0 |
| n  |  18 |   4 |   6 |   3 |  13 |   3 |   1 |   0 |  11 |   0 |   6 |   0 |  20 | 385 |   5 |   1 |   0 |   4 |  19 |  23 |   7 |   0 |   1 |   7 |   2 |   1 |
| o  |  10 |   9 |  23 |  11 |  21 |   9 |   7 |   3 |  17 |   0 |   3 |   3 |  22 |   8 | 328 |   4 |   1 |   8 |  14 |  23 |   4 |   0 |   3 |   1 |   8 |   0 |
| p  |   0 |   0 |   1 |   3 |   0 |   0 |   4 |   1 |   2 |   0 |   1 |   3 |   0 |   0 |   0 | 497 |   6 |   7 |   1 |   4 |   0 |   2 |   1 |   6 |   1 |   0 |
| q  |   0 |   0 |   6 |   1 |   0 |   1 |   1 |   1 |   2 |   0 |   1 |   0 |   0 |   0 |   0 |   1 | 501 |   5 |   0 |   3 |   9 |   1 |   2 |   3 |   2 |   0 |
| r  |   2 |   3 |  10 |   4 |   1 |   3 |  10 |   5 |   2 |   1 |   7 |   4 |   0 |   0 |   0 |   9 |   7 | 433 |   2 |   1 |   9 |   6 |   8 |   1 |  10 |   2 |
| s  |  29 |   5 |   2 |   4 |   3 |  14 |   3 |   4 |   4 |   0 |   4 |   3 |  12 |  12 |   5 |   0 |   2 |   6 | 397 |  12 |   3 |   2 |   1 |   7 |   6 |   0 |
| t  |  10 |   3 |   3 |   1 |   6 |   1 |   1 |   0 |   2 |   0 |   0 |   0 |   2 |   2 |   4 |   0 |   2 |   0 |   1 | 493 |   2 |   1 |   3 |   0 |   2 |   1 |
| u  |   0 |   6 |  15 |  24 |   2 |  11 |  11 |   5 |  17 |   0 |  16 |   7 |   4 |   0 |   1 |   0 |   2 |  21 |   4 |  13 | 351 |   8 |   5 |   2 |  15 |   0 |
| v  |   0 |   0 |   0 |   0 |   0 |   1 |   2 |   3 |   2 |   0 |   0 |   0 |   0 |   0 |   0 |  11 |   1 |   1 |   0 |   0 |   4 | 504 |  11 |   0 |   0 |   0 |
| w  |   0 |   0 |   1 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   4 |   2 |   0 |   0 |   0 |   0 |   1 | 532 |   0 |   0 |   0 |
| x  |   1 |   6 |  16 |   4 |  10 |   1 |   8 |   2 |   9 |   0 |   3 |  11 |   1 |   1 |   1 |   8 |   1 |  10 |   0 |   4 |  16 |   6 |   6 | 412 |   3 |   0 |
| y  |   7 |   0 |   5 |   0 |   4 |   0 |   2 |   1 |   4 |   0 |   0 |   1 |   0 |   1 |   2 |   0 |   1 |   4 |   1 |   7 |   2 |   1 |   0 |   1 | 496 |   0 |
| z  |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |


# Testing Statistics

In [97]:
total_samples = 0
total_misclass = 0
test_data_loader = DataLoader(test_asl_dataset, **params)
all_y = np.array([], dtype=np.uint8)
all_y_hat = np.array([], dtype=np.uint8)

if use_cuda:
    model = model.cuda()
for i, sample in enumerate(test_data_loader):
    y = sample['y']
    y = y.data.numpy()
#     images = (sample['image'])
#     images = images.float()
    images = (sample['image'])
    if use_cuda:
        images = images.cuda()
    images = images.float()
    
#     model = model.cpu()
    predictions = model(images)
    predictions = predictions.cpu()
    predictions = predictions.data.numpy()
    y_hat = np.argmax(predictions, axis=1)
    misclass = np.sum(np.where(y != y_hat, 1, 0))
    total_samples += y.shape[0]
    total_misclass += misclass
    all_y = np.append(all_y, y)
    all_y_hat = np.append(all_y_hat, y_hat)
    #print(f"all_y = {all_y}")
    #print(f"all_y_hat = {all_y_hat}")
    print(f"Number of Misclassifications = {misclass}")
    print(f"Sample acc = {(y.shape[0]-misclass)/y.shape[0]*100}")
overall_acc = (total_samples - total_misclass)/total_samples
print(f"Overall Accuracy = {overall_acc}")

C:\Users\wills\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Number of Misclassifications = 26
Sample acc = 45.83333333333333
Number of Misclassifications = 23
Sample acc = 52.083333333333336
Number of Misclassifications = 26
Sample acc = 45.83333333333333
Number of Misclassifications = 30
Sample acc = 37.5
Number of Misclassifications = 28
Sample acc = 41.66666666666667
Number of Misclassifications = 30
Sample acc = 37.5
Number of Misclassifications = 28
Sample acc = 41.66666666666667
Number of Misclassifications = 25
Sample acc = 47.91666666666667
Number of Misclassifications = 32
Sample acc = 33.33333333333333
Number of Misclassifications = 26
Sample acc = 45.83333333333333
Number of Misclassifications = 27
Sample acc = 43.75
Number of Misclassifications = 19
Sample acc = 60.416666666666664
Number of Misclassifications = 27
Sample acc = 43.75
Number of Misclassifications = 34
Sample acc = 29.166666666666668
Number of Misclassifications = 31
Sample acc = 35.41666666666667
Number of Misclassifications = 29
Sample acc = 39.58333333333333
Number 

In [116]:
cm_df, report = make_stats(all_y, all_y_hat, num_classes=24)
#print(cm_df.to_markdown())
cm_df.index = alphabet_reduced
cm_df.columns = alphabet_reduced
print(cm_df.to_markdown())
for cl, lt in enumerate(alphabet):
    cl_str = f' {cl} '
    lt_str = f' {lt} '
    lt_str = ' '*(len(cl_str) - len(lt_str)) + lt_str
#     print(cl_str, lt_str)
    report = report.replace(cl_str, lt_str)
print(report)
    
#     print(type(report))

|    |   a |   b |   c |   d |   e |   f |   g |   h |   i |   k |   l |   m |   n |   o |   p |   q |   r |   s |   t |   u |   v |   w |   x |   y |
|:---|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|
| a  |  73 |   5 |   0 |   2 |   7 |   3 |   2 |   2 |   3 |   0 |   1 |   8 |  15 |   8 |   1 |   3 |   2 |  13 |  19 |   5 |   1 |   0 |   5 |   2 |
| b  |   3 |  95 |   3 |   2 |   8 |  16 |  10 |  10 |   8 |   3 |   0 |   0 |   1 |   2 |   3 |   3 |   3 |   1 |   2 |   6 |   0 |   0 |   0 |   1 |
| c  |   1 |   9 |  76 |   2 |   4 |   7 |   1 |  17 |  14 |   7 |   1 |   4 |   0 |   1 |   1 |   9 |   2 |   6 |   5 |   3 |   0 |   1 |   1 |   8 |
| d  |   4 |   2 |   2 |  84 |   3 |   4 |   6 |   4 |   1 |   5 |  13 |   0 |   0 |   1 |   0 |   6 |   5 |   2 |   6 |   4 |  11 |   6 |   6 |   5 |
| e  |   7 |   3 |   2 |   3 |  61 |   4 |   1 |   3 |  17 |   0 |   2 |  23 |   9 |   9 |   0

|    |   a |   b |   c |   d |   e |   f |   g |   h |   i |   k |   l |   m |   n |   o |   p |   q |   r |   s |   t |   u |   v |   w |   x |   y |
|:---|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|
| a  |  73 |   5 |   0 |   2 |   7 |   3 |   2 |   2 |   3 |   0 |   1 |   8 |  15 |   8 |   1 |   3 |   2 |  13 |  19 |   5 |   1 |   0 |   5 |   2 |
| b  |   3 |  95 |   3 |   2 |   8 |  16 |  10 |  10 |   8 |   3 |   0 |   0 |   1 |   2 |   3 |   3 |   3 |   1 |   2 |   6 |   0 |   0 |   0 |   1 |
| c  |   1 |   9 |  76 |   2 |   4 |   7 |   1 |  17 |  14 |   7 |   1 |   4 |   0 |   1 |   1 |   9 |   2 |   6 |   5 |   3 |   0 |   1 |   1 |   8 |
| d  |   4 |   2 |   2 |  84 |   3 |   4 |   6 |   4 |   1 |   5 |  13 |   0 |   0 |   1 |   0 |   6 |   5 |   2 |   6 |   4 |  11 |   6 |   6 |   5 |
| e  |   7 |   3 |   2 |   3 |  61 |   4 |   1 |   3 |  17 |   0 |   2 |  23 |   9 |   9 |   0 |   1 |   3 |   6 |  21 |   1 |   0 |   1 |   1 |   2 |
| f  |   0 |  14 |   4 |   0 |  13 | 100 |   1 |   7 |   6 |   6 |   2 |   1 |   1 |   2 |   0 |   0 |   2 |  11 |   4 |   2 |   1 |   0 |   3 |   0 |
| g  |   5 |   4 |   7 |   9 |   2 |   4 |  58 |   6 |   6 |   5 |   2 |   1 |   3 |   2 |   7 |   1 |   7 |   1 |  17 |   6 |  16 |   3 |   5 |   3 |
| h  |   0 |   4 |   6 |   3 |   1 |   2 |  15 |  99 |   1 |   2 |   2 |   0 |   0 |   0 |   2 |   7 |   8 |   3 |   4 |   4 |   5 |   4 |   4 |   4 |
| i  |   1 |   7 |  12 |   0 |  21 |   1 |   3 |   1 |  83 |   3 |   1 |   9 |   4 |   4 |   0 |   1 |   3 |   4 |   8 |   3 |   1 |   2 |   2 |   6 |
| k  |   0 |   0 |   1 |   2 |   1 |   1 |   3 |   0 |   4 | 112 |   8 |   0 |   0 |   0 |   2 |   1 |   2 |   0 |   0 |   9 |  15 |  11 |   5 |   3 |
| l  |   0 |   2 |   0 |  10 |   0 |   0 |   2 |   2 |   0 |  21 |  90 |   0 |   1 |   0 |   1 |   2 |   6 |   0 |   2 |   6 |  20 |  11 |   1 |   3 |
| m  |  16 |   6 |   2 |   3 |   7 |   0 |   0 |   1 |  10 |   0 |   0 |  57 |  13 |  12 |   0 |   0 |   5 |  20 |  23 |   0 |   0 |   0 |   1 |   4 |
| n  |  16 |   1 |   2 |   1 |   5 |   1 |   2 |   0 |   2 |   4 |   0 |  12 |  66 |   8 |   1 |   1 |   5 |  18 |  20 |   6 |   0 |   1 |   4 |   4 |
| o  |  15 |  10 |  11 |   1 |   9 |   7 |   2 |   0 |  20 |   0 |   2 |  16 |   6 |  47 |   0 |   0 |   3 |  12 |  11 |   2 |   0 |   0 |   2 |   4 |
| p  |   0 |   2 |   1 |   4 |   0 |   0 |  15 |   3 |   2 |   3 |   1 |   0 |   0 |   0 |  88 |  23 |  19 |   1 |   3 |   1 |   5 |   6 |   2 |   1 |
| q  |   1 |   3 |   4 |   1 |   0 |   0 |   6 |   1 |   5 |   3 |   8 |   2 |   1 |   0 |   9 |  96 |   6 |   0 |   5 |   5 |   7 |   1 |  10 |   6 |
| r  |   5 |   4 |   6 |   2 |   0 |  10 |   4 |  14 |   0 |   6 |   4 |   0 |   1 |   2 |   9 |   5 |  61 |   1 |  11 |  17 |   3 |   8 |   5 |   2 |
| s  |  26 |   9 |   2 |   3 |   6 |   3 |   2 |   3 |   6 |   3 |   2 |  16 |  15 |   7 |   0 |   2 |   2 |  43 |  16 |   6 |   1 |   0 |   7 |   0 |
| t  |  10 |   3 |   1 |   1 |  10 |   1 |   1 |   2 |   5 |   1 |   6 |  13 |  10 |   3 |   3 |   5 |   2 |  18 |  66 |   6 |   0 |   5 |   4 |   4 |
| u  |   3 |   5 |   7 |   6 |   0 |   5 |  14 |   7 |   2 |   7 |   9 |   2 |   1 |   1 |   0 |   3 |  10 |   0 |   6 |  65 |   9 |   8 |   2 |   8 |
| v  |   0 |   0 |   1 |   7 |   0 |   0 |   3 |   1 |   0 |  11 |  10 |   0 |   0 |   0 |  10 |   0 |   2 |   0 |   0 |   4 | 110 |  17 |   3 |   1 |
| w  |   0 |   0 |   2 |   0 |   0 |   1 |   0 |   2 |   0 |   4 |   1 |   0 |   0 |   0 |   6 |   2 |   2 |   0 |   0 |   2 |  21 | 134 |   0 |   3 |
| x  |  11 |   3 |   6 |   8 |   2 |   0 |   8 |   0 |  13 |   2 |  10 |   0 |   2 |   0 |   4 |   3 |   9 |  10 |   4 |   6 |   8 |   7 |  63 |   1 |
| y  |  17 |   1 |  11 |   4 |   3 |   0 |   2 |   3 |  10 |   1 |   0 |   0 |   6 |   2 |   0 |   6 |   4 |   4 |  15 |   2 |   0 |   0 |   6 |  83 |

# Testing Loaded Model

## loaded training stats

In [101]:
total_samples = 0
total_misclass = 0
# test_data_loader = DataLoader(test_asl_dataset, **params)
all_y = np.array([], dtype=np.uint8)
all_y_hat = np.array([], dtype=np.uint8)

if use_cuda:
    loaded_model = loaded_model.cuda()
for i, sample in enumerate(data_loader):
    y = sample['y']
    y = y.data.numpy()
#     images = (sample['image'])
#     images = images.float()
    images = (sample['image'])
    if use_cuda:
        images = images.cuda()
    images = images.float()
    
#     model = model.cpu()
    predictions = loaded_model(images)
    predictions = predictions.cpu()
    predictions = predictions.data.numpy()
    y_hat = np.argmax(predictions, axis=1)
    misclass = np.sum(np.where(y != y_hat, 1, 0))
    total_samples += y.shape[0]
    total_misclass += misclass
    all_y = np.append(all_y, y)
    all_y_hat = np.append(all_y_hat, y_hat)
    #print(f"all_y = {all_y}")
    #print(f"all_y_hat = {all_y_hat}")
    print(f"Number of Misclassifications = {misclass}")
    print(f"Sample acc = {(y.shape[0]-misclass)/y.shape[0]*100}")
overall_acc = (total_samples - total_misclass)/total_samples
print(f"Overall Accuracy = {overall_acc}")

C:\Users\wills\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Number of Misclassifications = 11
Sample acc = 77.08333333333334
Number of Misclassifications = 7
Sample acc = 85.41666666666666
Number of Misclassifications = 6
Sample acc = 87.5
Number of Misclassifications = 3
Sample acc = 93.75
Number of Misclassifications = 5
Sample acc = 89.58333333333334
Number of Misclassifications = 7
Sample acc = 85.41666666666666
Number of Misclassifications = 4
Sample acc = 91.66666666666666
Number of Misclassifications = 6
Sample acc = 87.5
Number of Misclassifications = 7
Sample acc = 85.41666666666666
Number of Misclassifications = 7
Sample acc = 85.41666666666666
Number of Misclassifications = 6
Sample acc = 87.5
Number of Misclassifications = 10
Sample acc = 79.16666666666666
Number of Misclassifications = 11
Sample acc = 77.08333333333334
Number of Misclassifications = 11
Sample acc = 77.08333333333334
Number of Misclassifications = 7
Sample acc = 85.41666666666666
Number of Misclassifications = 8
Sample acc = 83.33333333333334
Number of Misclassifica

In [103]:
cm_df, report = make_stats(all_y, all_y_hat, num_classes=26)
#print(cm_df.to_markdown())
cm_df.index = alphabet
cm_df.columns = alphabet
print(cm_df.to_markdown())
for cl, lt in enumerate(alphabet):
    cl_str = f' {cl} '
    lt_str = f' {lt} '
    lt_str = ' '*(len(cl_str) - len(lt_str)) + lt_str
    report = report.replace(cl_str, lt_str)
print(report)
    
#     print(type(report))

|    |   a |   b |   c |   d |   e |   f |   g |   h |   i |   j |   k |   l |   m |   n |   o |   p |   q |   r |   s |   t |   u |   v |   w |   x |   y |   z |
|:---|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|
| a  | 487 |   2 |   1 |   7 |   2 |   4 |   1 |   1 |   1 |   0 |   0 |   0 |  15 |   4 |   4 |   0 |   0 |   0 |   1 |   4 |   0 |   4 |   0 |   0 |   2 |   0 |
| b  |   4 | 472 |   3 |   3 |   0 |  17 |   4 |   5 |   0 |   0 |   0 |   0 |   2 |   0 |   1 |   6 |   7 |   1 |   0 |   6 |   2 |   3 |   0 |   1 |   3 |   0 |
| c  |   1 |   4 | 487 |   1 |   0 |   0 |   4 |  12 |   2 |   0 |   3 |   0 |   0 |   1 |   0 |   3 |   6 |   7 |   2 |   0 |   3 |   0 |   0 |   3 |   1 |   0 |
| d  |   2 |   0 |   5 | 456 |   4 |   1 |  12 |   2 |   3 |   0 |   5 |   9 |   0 |   0 |   0 |   6 |   6 |   6 |   3 |   5 |   0 |   9 |   1 |   0 |   5 |   0 |
| e  |  19 |   0 |   1

C:\Users\wills\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\wills\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\wills\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r

|    |   a |   b |   c |   d |   e |   f |   g |   h |   i |   j |   k |   l |   m |   n |   o |   p |   q |   r |   s |   t |   u |   v |   w |   x |   y |   z |
|:---|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|
| a  | 487 |   2 |   1 |   7 |   2 |   4 |   1 |   1 |   1 |   0 |   0 |   0 |  15 |   4 |   4 |   0 |   0 |   0 |   1 |   4 |   0 |   4 |   0 |   0 |   2 |   0 |
| b  |   4 | 472 |   3 |   3 |   0 |  17 |   4 |   5 |   0 |   0 |   0 |   0 |   2 |   0 |   1 |   6 |   7 |   1 |   0 |   6 |   2 |   3 |   0 |   1 |   3 |   0 |
| c  |   1 |   4 | 487 |   1 |   0 |   0 |   4 |  12 |   2 |   0 |   3 |   0 |   0 |   1 |   0 |   3 |   6 |   7 |   2 |   0 |   3 |   0 |   0 |   3 |   1 |   0 |
| d  |   2 |   0 |   5 | 456 |   4 |   1 |  12 |   2 |   3 |   0 |   5 |   9 |   0 |   0 |   0 |   6 |   6 |   6 |   3 |   5 |   0 |   9 |   1 |   0 |   5 |   0 |
| e  |  19 |   0 |   1 |   4 | 382 |   3 |   2 |   0 |  12 |   0 |   0 |   0 |  31 |  12 |   9 |   0 |   3 |   9 |  12 |  32 |   3 |   0 |   1 |   2 |   3 |   0 |
| f  |   1 |   1 |   2 |   0 |   5 | 457 |   2 |   5 |   1 |   0 |   4 |   3 |   7 |   1 |   3 |   0 |   3 |   0 |  16 |  24 |   1 |   0 |   1 |   1 |   2 |   0 |
| g  |   6 |   7 |  14 |   7 |   5 |   7 | 417 |   8 |   2 |   0 |   4 |   1 |   0 |   1 |   1 |  16 |   0 |  10 |   0 |   5 |   0 |   5 |   7 |   3 |  14 |   0 |
| h  |   0 |   0 |   1 |   1 |   0 |   0 |   5 | 510 |   1 |   1 |   0 |   0 |   1 |   1 |   0 |   2 |   1 |   4 |   4 |   0 |   3 |   2 |   0 |   3 |   0 |   0 |
| i  |   1 |   4 |   1 |   0 |   5 |   0 |   0 |   0 | 506 |   0 |   1 |   2 |   6 |   2 |   1 |   0 |   0 |   0 |   1 |   6 |   1 |   0 |   1 |   0 |   2 |   0 |
| j  |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |
| k  |   0 |   2 |  12 |   0 |   0 |   0 |   1 |   0 |   2 |   0 | 481 |   0 |   0 |   1 |   0 |   2 |   1 |   3 |   0 |   0 |   4 |   4 |   9 |   3 |  15 |   0 |
| l  |   0 |   1 |   2 |   1 |   0 |   0 |   1 |   0 |   1 |   0 |   3 | 519 |   0 |   0 |   0 |   0 |   0 |   2 |   0 |   2 |   2 |   2 |   2 |   2 |   0 |   0 |
| m  |   6 |   6 |   0 |   0 |   7 |   1 |   2 |   1 |   8 |   0 |   2 |   1 | 424 |  24 |   6 |   2 |   0 |   3 |  19 |  23 |   1 |   0 |   0 |   4 |   0 |   0 |
| n  |  18 |   4 |   6 |   3 |  13 |   3 |   1 |   0 |  11 |   0 |   6 |   0 |  20 | 385 |   5 |   1 |   0 |   4 |  19 |  23 |   7 |   0 |   1 |   7 |   2 |   1 |
| o  |  10 |   9 |  23 |  11 |  21 |   9 |   7 |   3 |  17 |   0 |   3 |   3 |  22 |   8 | 328 |   4 |   1 |   8 |  14 |  23 |   4 |   0 |   3 |   1 |   8 |   0 |
| p  |   0 |   0 |   1 |   3 |   0 |   0 |   4 |   1 |   2 |   0 |   1 |   3 |   0 |   0 |   0 | 497 |   6 |   7 |   1 |   4 |   0 |   2 |   1 |   6 |   1 |   0 |
| q  |   0 |   0 |   6 |   1 |   0 |   1 |   1 |   1 |   2 |   0 |   1 |   0 |   0 |   0 |   0 |   1 | 501 |   5 |   0 |   3 |   9 |   1 |   2 |   3 |   2 |   0 |
| r  |   2 |   3 |  10 |   4 |   1 |   3 |  10 |   5 |   2 |   1 |   7 |   4 |   0 |   0 |   0 |   9 |   7 | 433 |   2 |   1 |   9 |   6 |   8 |   1 |  10 |   2 |
| s  |  29 |   5 |   2 |   4 |   3 |  14 |   3 |   4 |   4 |   0 |   4 |   3 |  12 |  12 |   5 |   0 |   2 |   6 | 397 |  12 |   3 |   2 |   1 |   7 |   6 |   0 |
| t  |  10 |   3 |   3 |   1 |   6 |   1 |   1 |   0 |   2 |   0 |   0 |   0 |   2 |   2 |   4 |   0 |   2 |   0 |   1 | 493 |   2 |   1 |   3 |   0 |   2 |   1 |
| u  |   0 |   6 |  15 |  24 |   2 |  11 |  11 |   5 |  17 |   0 |  16 |   7 |   4 |   0 |   1 |   0 |   2 |  21 |   4 |  13 | 351 |   8 |   5 |   2 |  15 |   0 |
| v  |   0 |   0 |   0 |   0 |   0 |   1 |   2 |   3 |   2 |   0 |   0 |   0 |   0 |   0 |   0 |  11 |   1 |   1 |   0 |   0 |   4 | 504 |  11 |   0 |   0 |   0 |
| w  |   0 |   0 |   1 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   4 |   2 |   0 |   0 |   0 |   0 |   1 | 532 |   0 |   0 |   0 |
| x  |   1 |   6 |  16 |   4 |  10 |   1 |   8 |   2 |   9 |   0 |   3 |  11 |   1 |   1 |   1 |   8 |   1 |  10 |   0 |   4 |  16 |   6 |   6 | 412 |   3 |   0 |
| y  |   7 |   0 |   5 |   0 |   4 |   0 |   2 |   1 |   4 |   0 |   0 |   1 |   0 |   1 |   2 |   0 |   1 |   4 |   1 |   7 |   2 |   1 |   0 |   1 | 496 |   0 |
| z  |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |   0 |

# Loaded testing stats

In [104]:
total_samples = 0
total_misclass = 0
test_data_loader = DataLoader(test_asl_dataset, **params)
all_y = np.array([], dtype=np.uint8)
all_y_hat = np.array([], dtype=np.uint8)

if use_cuda:
    loaded_model = loaded_model.cuda()
for i, sample in enumerate(test_data_loader):
    y = sample['y']
    y = y.data.numpy()
#     images = (sample['image'])
#     images = images.float()
    images = (sample['image'])
    if use_cuda:
        images = images.cuda()
    images = images.float()
    
#     model = model.cpu()
    predictions = loaded_model(images)
    predictions = predictions.cpu()
    predictions = predictions.data.numpy()
    y_hat = np.argmax(predictions, axis=1)
    misclass = np.sum(np.where(y != y_hat, 1, 0))
    total_samples += y.shape[0]
    total_misclass += misclass
    all_y = np.append(all_y, y)
    all_y_hat = np.append(all_y_hat, y_hat)
    #print(f"all_y = {all_y}")
    #print(f"all_y_hat = {all_y_hat}")
    print(f"Number of Misclassifications = {misclass}")
    print(f"Sample acc = {(y.shape[0]-misclass)/y.shape[0]*100}")
overall_acc = (total_samples - total_misclass)/total_samples
print(f"Overall Accuracy = {overall_acc}")

C:\Users\wills\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Number of Misclassifications = 25
Sample acc = 47.91666666666667
Number of Misclassifications = 37
Sample acc = 22.916666666666664
Number of Misclassifications = 22
Sample acc = 54.166666666666664
Number of Misclassifications = 23
Sample acc = 52.083333333333336
Number of Misclassifications = 29
Sample acc = 39.58333333333333
Number of Misclassifications = 32
Sample acc = 33.33333333333333
Number of Misclassifications = 23
Sample acc = 52.083333333333336
Number of Misclassifications = 29
Sample acc = 39.58333333333333
Number of Misclassifications = 25
Sample acc = 47.91666666666667
Number of Misclassifications = 29
Sample acc = 39.58333333333333
Number of Misclassifications = 28
Sample acc = 41.66666666666667
Number of Misclassifications = 27
Sample acc = 43.75
Number of Misclassifications = 23
Sample acc = 52.083333333333336
Number of Misclassifications = 22
Sample acc = 54.166666666666664
Number of Misclassifications = 25
Sample acc = 47.91666666666667
Number of Misclassifications = 

In [118]:
cm_df, report = make_stats(all_y, all_y_hat, num_classes=24)
#print(cm_df.to_markdown())
cm_df.index = alphabet_reduced
cm_df.columns = alphabet_reduced
print(cm_df.to_markdown())
for cl, lt in enumerate(alphabet):
    cl_str = f' {cl} '
    lt_str = f' {lt} '
    lt_str = ' '*(len(cl_str) - len(lt_str)) + lt_str
    report = report.replace(cl_str, lt_str)
print(report)
    
#     print(type(report))

|    |   a |   b |   c |   d |   e |   f |   g |   h |   i |   k |   l |   m |   n |   o |   p |   q |   r |   s |   t |   u |   v |   w |   x |   y |
|:---|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|
| a  |  73 |   5 |   0 |   2 |   7 |   3 |   2 |   2 |   3 |   0 |   1 |   8 |  15 |   8 |   1 |   3 |   2 |  13 |  19 |   5 |   1 |   0 |   5 |   2 |
| b  |   3 |  95 |   3 |   2 |   8 |  16 |  10 |  10 |   8 |   3 |   0 |   0 |   1 |   2 |   3 |   3 |   3 |   1 |   2 |   6 |   0 |   0 |   0 |   1 |
| c  |   1 |   9 |  76 |   2 |   4 |   7 |   1 |  17 |  14 |   7 |   1 |   4 |   0 |   1 |   1 |   9 |   2 |   6 |   5 |   3 |   0 |   1 |   1 |   8 |
| d  |   4 |   2 |   2 |  84 |   3 |   4 |   6 |   4 |   1 |   5 |  13 |   0 |   0 |   1 |   0 |   6 |   5 |   2 |   6 |   4 |  11 |   6 |   6 |   5 |
| e  |   7 |   3 |   2 |   3 |  61 |   4 |   1 |   3 |  17 |   0 |   2 |  23 |   9 |   9 |   0

|    |   a |   b |   c |   d |   e |   f |   g |   h |   i |   k |   l |   m |   n |   o |   p |   q |   r |   s |   t |   u |   v |   w |   x |   y |
|:---|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|----:|
| a  |  73 |   5 |   0 |   2 |   7 |   3 |   2 |   2 |   3 |   0 |   1 |   8 |  15 |   8 |   1 |   3 |   2 |  13 |  19 |   5 |   1 |   0 |   5 |   2 |
| b  |   3 |  95 |   3 |   2 |   8 |  16 |  10 |  10 |   8 |   3 |   0 |   0 |   1 |   2 |   3 |   3 |   3 |   1 |   2 |   6 |   0 |   0 |   0 |   1 |
| c  |   1 |   9 |  76 |   2 |   4 |   7 |   1 |  17 |  14 |   7 |   1 |   4 |   0 |   1 |   1 |   9 |   2 |   6 |   5 |   3 |   0 |   1 |   1 |   8 |
| d  |   4 |   2 |   2 |  84 |   3 |   4 |   6 |   4 |   1 |   5 |  13 |   0 |   0 |   1 |   0 |   6 |   5 |   2 |   6 |   4 |  11 |   6 |   6 |   5 |
| e  |   7 |   3 |   2 |   3 |  61 |   4 |   1 |   3 |  17 |   0 |   2 |  23 |   9 |   9 |   0 |   1 |   3 |   6 |  21 |   1 |   0 |   1 |   1 |   2 |
| f  |   0 |  14 |   4 |   0 |  13 | 100 |   1 |   7 |   6 |   6 |   2 |   1 |   1 |   2 |   0 |   0 |   2 |  11 |   4 |   2 |   1 |   0 |   3 |   0 |
| g  |   5 |   4 |   7 |   9 |   2 |   4 |  58 |   6 |   6 |   5 |   2 |   1 |   3 |   2 |   7 |   1 |   7 |   1 |  17 |   6 |  16 |   3 |   5 |   3 |
| h  |   0 |   4 |   6 |   3 |   1 |   2 |  15 |  99 |   1 |   2 |   2 |   0 |   0 |   0 |   2 |   7 |   8 |   3 |   4 |   4 |   5 |   4 |   4 |   4 |
| i  |   1 |   7 |  12 |   0 |  21 |   1 |   3 |   1 |  83 |   3 |   1 |   9 |   4 |   4 |   0 |   1 |   3 |   4 |   8 |   3 |   1 |   2 |   2 |   6 |
| k  |   0 |   0 |   1 |   2 |   1 |   1 |   3 |   0 |   4 | 112 |   8 |   0 |   0 |   0 |   2 |   1 |   2 |   0 |   0 |   9 |  15 |  11 |   5 |   3 |
| l  |   0 |   2 |   0 |  10 |   0 |   0 |   2 |   2 |   0 |  21 |  90 |   0 |   1 |   0 |   1 |   2 |   6 |   0 |   2 |   6 |  20 |  11 |   1 |   3 |
| m  |  16 |   6 |   2 |   3 |   7 |   0 |   0 |   1 |  10 |   0 |   0 |  57 |  13 |  12 |   0 |   0 |   5 |  20 |  23 |   0 |   0 |   0 |   1 |   4 |
| n  |  16 |   1 |   2 |   1 |   5 |   1 |   2 |   0 |   2 |   4 |   0 |  12 |  66 |   8 |   1 |   1 |   5 |  18 |  20 |   6 |   0 |   1 |   4 |   4 |
| o  |  15 |  10 |  11 |   1 |   9 |   7 |   2 |   0 |  20 |   0 |   2 |  16 |   6 |  47 |   0 |   0 |   3 |  12 |  11 |   2 |   0 |   0 |   2 |   4 |
| p  |   0 |   2 |   1 |   4 |   0 |   0 |  15 |   3 |   2 |   3 |   1 |   0 |   0 |   0 |  88 |  23 |  19 |   1 |   3 |   1 |   5 |   6 |   2 |   1 |
| q  |   1 |   3 |   4 |   1 |   0 |   0 |   6 |   1 |   5 |   3 |   8 |   2 |   1 |   0 |   9 |  96 |   6 |   0 |   5 |   5 |   7 |   1 |  10 |   6 |
| r  |   5 |   4 |   6 |   2 |   0 |  10 |   4 |  14 |   0 |   6 |   4 |   0 |   1 |   2 |   9 |   5 |  61 |   1 |  11 |  17 |   3 |   8 |   5 |   2 |
| s  |  26 |   9 |   2 |   3 |   6 |   3 |   2 |   3 |   6 |   3 |   2 |  16 |  15 |   7 |   0 |   2 |   2 |  43 |  16 |   6 |   1 |   0 |   7 |   0 |
| t  |  10 |   3 |   1 |   1 |  10 |   1 |   1 |   2 |   5 |   1 |   6 |  13 |  10 |   3 |   3 |   5 |   2 |  18 |  66 |   6 |   0 |   5 |   4 |   4 |
| u  |   3 |   5 |   7 |   6 |   0 |   5 |  14 |   7 |   2 |   7 |   9 |   2 |   1 |   1 |   0 |   3 |  10 |   0 |   6 |  65 |   9 |   8 |   2 |   8 |
| v  |   0 |   0 |   1 |   7 |   0 |   0 |   3 |   1 |   0 |  11 |  10 |   0 |   0 |   0 |  10 |   0 |   2 |   0 |   0 |   4 | 110 |  17 |   3 |   1 |
| w  |   0 |   0 |   2 |   0 |   0 |   1 |   0 |   2 |   0 |   4 |   1 |   0 |   0 |   0 |   6 |   2 |   2 |   0 |   0 |   2 |  21 | 134 |   0 |   3 |
| x  |  11 |   3 |   6 |   8 |   2 |   0 |   8 |   0 |  13 |   2 |  10 |   0 |   2 |   0 |   4 |   3 |   9 |  10 |   4 |   6 |   8 |   7 |  63 |   1 |
| y  |  17 |   1 |  11 |   4 |   3 |   0 |   2 |   3 |  10 |   1 |   0 |   0 |   6 |   2 |   0 |   6 |   4 |   4 |  15 |   2 |   0 |   0 |   6 |  83 |